EfficientNet + Residual Connection + Transformer Encoder + CBAM

In [ ]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from efficientnet_pytorch import EfficientNet

seed = 2021
deterministic = True

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

# Early Stopping 클래스
class EarlyStopping:
    def __init__(self, patience=10, delta=0.001):
        self.patience = patience
        self.delta = delta
        self.best_loss = None
        self.counter = 0
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None or val_loss < self.best_loss - self.delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

# ==============================
# Residual Block (잔차 연결)
# ==============================
class ResidualBlock(nn.Module):
    def __init__(self, in_channels):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, in_channels, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(in_channels)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(in_channels, in_channels, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(in_channels)

    def forward(self, x):
        identity = x  # 입력값 저장 (잔차 연결)
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out += identity  # Residual Connection
        out = self.relu(out)
        return out

class CBAM(nn.Module):
    def __init__(self, in_channels, reduction=16):
        super(CBAM, self).__init__()

        # 채널 어텐션
        self.channel_att = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(in_channels, in_channels // reduction, kernel_size=1),
            nn.ReLU(),
            nn.Conv2d(in_channels // reduction, in_channels, kernel_size=1),
            nn.Sigmoid()
        )

        # 공간 어텐션
        self.spatial_att = nn.Sequential(
            nn.Conv2d(in_channels, 1, kernel_size=7, padding=3),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = x * self.channel_att(x)
        x = x * self.spatial_att(x)
        return x

class TransformerEncoder(nn.Module):
    def __init__(self, in_features, num_heads=4, dim_feedforward=512, num_layers=2):
        super(TransformerEncoder, self).__init__()
        self.encoder_layer = nn.TransformerEncoderLayer(
            d_model=in_features,
            nhead=num_heads,
            dim_feedforward=dim_feedforward,
            dropout=0.1
        )
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)

    def forward(self, x):
        x = x.flatten(2).permute(2, 0, 1)  # (batch, C, H, W) → (H*W, batch, C)
        x = self.transformer_encoder(x)
        x = x.mean(dim=0)  # 평균 풀링
        return x

class CustomEfficientNet(nn.Module):
    def __init__(self, model_name="efficientnet-b0", num_classes=6, freeze_backbone=False):
        super(CustomEfficientNet, self).__init__()
        self.backbone = EfficientNet.from_pretrained(model_name)
        if freeze_backbone:
            for param in self.backbone.parameters():
                param.requires_grad = False

        in_features = self.backbone._fc.in_features
        self.backbone._fc = nn.Identity()

        # Residual Block 추가
        self.residual_block = ResidualBlock(in_channels=in_features)

        # CBAM 블록 추가
        self.cbam = CBAM(in_channels=in_features)

        # Transformer Encoder 추가
        self.transformer = TransformerEncoder(in_features=in_features)

        # 분류기
        self.classifier = nn.Sequential(
            nn.Linear(in_features, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.backbone.extract_features(x)  # EfficientNet 특징 추출
        x = self.residual_block(x)  # Residual Block 적용
        x = self.cbam(x)  # CBAM 적용
        x = self.transformer(x)  # Transformer Encoder 적용
        x = self.classifier(x)
        return x

# 하이퍼파라미터
model_name = 'efficientnet-b0'
num_classes = 6
batch_size = 32
epochs = 50
learning_rate = 0.001
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 데이터 전처리 & 로더
image_size = EfficientNet.get_image_size(model_name)
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
dataset_path = 'C:/Users/IIALAB/Desktop/kdm/solar/kaggle/input/solar-panel-images/Faulty_solar_panel'
dataset = datasets.ImageFolder(root=dataset_path, transform=transform)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# 모델 및 손실 함수, 최적화기
model = CustomEfficientNet(model_name=model_name, num_classes=num_classes, freeze_backbone=False).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 학습 및 평가 함수
def train(model, loader, criterion, optimizer):
    model.train()
    running_loss, correct, total = 0.0, 0, 0
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)
        _, pred = outputs.max(1)
        correct += (pred == labels).sum().item()
        total += labels.size(0)
    return running_loss/total, correct/total

def evaluate(model, loader, criterion):
    model.eval()
    running_loss, correct, total = 0.0, 0, 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * images.size(0)
            _, pred = outputs.max(1)
            correct += (pred == labels).sum().item()
            total += labels.size(0)
    return running_loss/total, correct/total

# 학습 루프
best_test_acc = 0.0
best_epoch = 0
early_stopping = EarlyStopping(patience=10)

for epoch in range(epochs):
    train_loss, train_acc = train(model, train_loader, criterion, optimizer)
    test_loss, test_acc = evaluate(model, test_loader, criterion)

    if test_acc > best_test_acc:
        best_test_acc = test_acc
        best_epoch = epoch + 1

    early_stopping(test_loss)
    if early_stopping.early_stop:
        print("Early stopping triggered")
        break

    print(f"[Epoch {epoch+1}/{epochs}] Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
    print(f"  Test Loss: {test_loss:.4f}  | Test Acc: {test_acc:.4f}")

print(f"\nTraining complete! Best Test Accuracy: {best_test_acc:.4f} at Epoch {best_epoch}")

